**LIBRERIAS**

In [1]:
import pandas as pd

**ABRIMOS LOS ARCHIVOS NECESARIOS**

In [13]:
df_games = pd.read_csv(f'archivos definitivos/games.csv')
df_items = pd.read_csv(f'archivos definitivos/items.csv')
df_reviews = pd.read_csv(f'archivos definitivos/reviews.csv')

In [14]:
df_reviews

,item_id,user_id,year,recommend,sentiment_analysis
0,1250,76561197970982479,2011,True,2
1,22200,76561197970982479,2011,True,1
2,43110,76561197970982479,2011,True,2
3,251610,js41637,2014,True,2
4,227300,js41637,2013,True,2
...,...,...,...,...,...
59300,70,76561198312638244,2014,True,2
59301,362890,76561198312638244,2014,True,2
59302,273110,LydiaMorley,2014,True,2
59303,730,LydiaMorley,2014,True,2


**PRIMERA FUNCION**

In [71]:
def PlayTimeGenre( genero : str ):
    # Hacemos un join de las tablas para tener las horas agrupadas a los items
    df_join = df_games.merge(df_items, on='item_id', how='inner')

    # Filtro por genero solicitado
    df_join = df_join.loc[df_join['genres']== genero].drop(columns=['release_date'], axis=1)

    # Realizo la suma por año 
    suma_por_años = df_join.groupby('year')['playtime_forever'].sum()

    # Escojemos el año con la suma más alta
    año_con_mas_horas = suma_por_años.idxmax()

    # Retorno el año con mas horas por genero
    return f"El año de lanzamiento con más horas jugadas para el Género {genero} es {año_con_mas_horas}"

In [73]:
#Al ponerle el csv de items con 3 columnas el tiempo de retorno es de 3.4s, anteriormente con el csv de 2 columnas era de 2.2s. Ahora demora un 55% más que antes.

In [72]:
PlayTimeGenre('Action')

'El año de lanzamiento con más horas jugadas para el Género Action es 2012'

**SEGUNDA FUNCION**

In [212]:
def UserForGenre( genero : str ):
    # Hacemos un join de las tablas para tener las horas, el usuario agrupadas a los items
    df_join = df_games.merge(df_items, on='item_id', how='inner')

    # Filtro por el genero solicitado
    df_join = df_join.loc[df_join['genres']== genero].drop(columns=['release_date'], axis=1)

    # Sumo por usuario (ya esta filtrado por el genero). Me crea un df
    suma_por_años = df_join.groupby(['user_id'])['playtime_forever'].sum().reset_index()
    
    # Guardo el nombre del usuario con más horas jugadas en una variable
    user_con_mas_horas = suma_por_años.max()[0]

    # Creo un nuevo df en donde filtro por el usuario con más horas, para poder sacar sus horas por año
    df_join_user = df_join[df_join['user_id'] == user_con_mas_horas].drop(columns=['item_id','user_id','genres'], axis=1)

    # Ahora voy a sumar por año (ya esta filtrado por el usuario). Me crea un df
    suma_por_años = df_join_user.groupby('year')['playtime_forever'].sum().reset_index()

    # Conveirto el DataFrame a un diccionario para poder iterarlo
    suma_por_años_dic = suma_por_años.to_dict(orient='records')

    # Crear una lista para poder darle una forma más clara a la respuesta
    año_y_horas = [(f"| En el año {i['year']} jugó {i['playtime_forever']} horas |") for i in suma_por_años_dic]

    # Retorno la respuesta
    return (f"""El usuario con más horas jugadas para el género {genero} es {user_con_mas_horas}. Su historial es:
            {año_y_horas}""")


In [213]:
UserForGenre('Action')

"El usuario con más horas jugadas para el género Action es zzzmidmiss. Su historial es:\n            ['| En el año 2009 jugó 482 horas |', '| En el año 2010 jugó 273 horas |', '| En el año 2011 jugó 2302 horas |', '| En el año 2012 jugó 5714 horas |', '| En el año 2013 jugó 98 horas |', '| En el año 2014 jugó 16 horas |']"

**TERCERA FUNCION**

In [261]:
def UsersRecommend(año : int):
    #filtramos el df por el año, es decir unicamente queremos datos que correspondan a ese año
    df_reviews_filtrado = df_reviews[df_reviews['year'] == año]

    #Filtramos el df que ya esta filtrado, por el analisis sentiment sea 1 o 2
    df_reviews_filtrado = df_reviews_filtrado[df_reviews_filtrado['sentiment_analysis'].isin([1, 2])]

    #Agrupamos por item_id, el conteo de las recomendaciones True
    conteo_recomendacion_por_juego = df_reviews_filtrado.groupby('item_id')['recommend'].sum(True).reset_index()

    #Hacemos un df en donde ponemos los 3 con más conteo
    top_3_recommended = conteo_recomendacion_por_juego.nlargest(3, 'recommend')

    #Retornamos el Id de los juegos con más recomendaciones por el año
    return f'Id de los 3 juegos más recomendados por el año {año}: | Puesto 1 = {top_3_recommended["item_id"].iloc[0]} | Puesto 2 = {top_3_recommended["item_id"].iloc[1]} | Puesto 3 = {top_3_recommended["item_id"].iloc[2]} |'

In [262]:
UsersRecommend(2014)

'Id de los 3 juegos más recomendados por el año 2014: | Puesto 1 = 440 | Puesto 2 = 730 | Puesto 3 = 4000 |'

**CUARTA FUNCION**

In [282]:
def UsersNotRecommend( año : int ):
    #filtramos el df por el año, es decir unicamente queremos datos que correspondan a ese año
    df_reviews_filtrado = df_reviews[df_reviews['year'] == año]

    #Filtramos el df que ya esta filtrado, por el analisis sentiment sea 0
    df_reviews_filtrado = df_reviews_filtrado[df_reviews_filtrado['sentiment_analysis'].isin([0])]

    #Agrupamos por item_id, el conteo de las recomendaciones False, para esto usamos una funcion lambda en donde usamos el '~', que se lo usa para negar el True, ya que no es correcto usar .sum(False), entonces utilizamos la negacion de True(No True) que es equivalente a False
    conteo_recomendacion_por_juego = df_reviews_filtrado.groupby('item_id')['recommend'].apply(lambda x: (~x).sum()).reset_index()

    #Hacemos un df en donde ponemos los 3 con más conteo de False
    top_3_recommended = conteo_recomendacion_por_juego.nlargest(3, 'recommend')

    #Retornamos el Id de los juegos con menos recomendaciones por el año
    return f'Id de los 3 juegos menos recomendados por el año {año}: | Puesto 1 = {top_3_recommended["item_id"].iloc[0]} | Puesto 2 = {top_3_recommended["item_id"].iloc[1]} | Puesto 3 = {top_3_recommended["item_id"].iloc[2]} |'

In [283]:
UsersNotRecommend(2014)

'Id de los 3 juegos menos recomendados por el año 2014: | Puesto 1 = 221100 | Puesto 2 = 211420 | Puesto 3 = 252490 |'

**QUINTA FUNCION**

In [330]:
def sentiment_analysis( año : int ):
    #La funcion debe estar según el año de lanzamiento, primero filtramos el df por el año
    df_games_filtrado_año = df_games[df_games['year_release'] == año]

    #Hacemos un inner join con dos df, el de 'games' que ya esta filtrado por la fecha que necesitamos y el df de 'reviews' de cual necesitamos la informacion del sentiment_analysis
    df_join_analysis = df_games_filtrado_año.merge(df_reviews, on='item_id', how='inner')

    #Quitamos columas innesesarias que no utilizaremos
    df_join_analysis = df_join_analysis[['item_id', 'year_release', 'user_id','sentiment_analysis']]

    #Hacemos un conteo de sentiment_analysis
    conteo_sentiment_analysis = df_join_analysis['sentiment_analysis'].value_counts()

    #Retornamos la cantidad de registros de reseñas de usuarios, ya sea negativo, neutro o positivo por el año de lanzamiento del juego
    return f'La cantidad de registros de reseñas de usuarios por el año de lanzamiento {año} son: Negativos = {conteo_sentiment_analysis[0]}, Neutros = {conteo_sentiment_analysis[1]}, Positivos = {conteo_sentiment_analysis[2]}'

In [331]:
sentiment_analysis(2014)

'La cantidad de registros de reseñas de usuarios por el año de lanzamiento 2014 son: Negativos = 215, Neutros = 7078, Positivos = 10572'